# INFO DASHBOARD

En este documento se encuentran los gráficos que se van a mostrar en el Dashboard final. Estos gráficos mostrarán información extra sobre los alojamientos de Airbnb de NY:
   * Pie chart con tipos de casa
   * Distribucion de precios (apilado barrios) - dinamico
   * Wordcloud features más repetidas
   * Response time y superhosts
   * Tabla: accomodates, beds, bathrooms
   * Occupancy rate (mapa o grafico??)
   * Tabla hosts top10 → % de casas sobre total NY

Todos estos datos serán actualizados con los siguientes filtros que se insertarán en el dashboard final (Dash)
  1. Tipo de casa
  2. Barrio grande o todo NY
  3. Slider con rango de precios
  4. Nota de reseña

## Librerías

In [1]:
import json
import math
from itertools import count
from urllib.parse import urlencode

import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
from cv2 import mean
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, auc, classification_report,
                             confusion_matrix, roc_auc_score, roc_curve)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from wordcloud import STOPWORDS, WordCloud

pio.renderers.default = 'vscode'
pio.templates.default = 'plotly'

## Importar datos ya filtrados

El filtrado y preprocesado de la información está en el archivo:

*Actual: dash_graphs_info.ipynb*

Import listings

In [2]:
listings_filtered_df = pd.read_parquet("../Data/total data/modified data/listings_processed.parquet")
pd.options.display.max_columns = None
listings_filtered_df.head()

,id,last_scraped,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,years_permanency,baths
0,2539,2022-09-07,2787,John,2008-09-07,within an hour,100,90,f,Gravesend,9.0,12.0,t,t,Kensington,Brooklyn,40.64529,-73.97238,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,"[""First aid kit"", ""Microwave"", ""Stove"", ""Coffe...",299.0,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,21,51,81,356,9,0,0,2015-12-04,2018-10-19,4.89,4.88,5.00,5.00,5.00,4.75,4.88,None,f,9,1,6,2,0.11,14.219178,1.0
1,2595,2022-09-07,2845,Jennifer,2008-09-09,within a day,75,23,f,Midtown,6.0,9.0,t,t,Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,"[""Stove"", ""Coffee maker"", ""Long term stays all...",175.0,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,0,0,5,280,49,1,0,2009-11-21,2022-06-21,4.68,4.73,4.63,4.77,4.80,4.81,4.40,None,f,3,3,0,0,0.31,14.213699,1.0
2,5121,2022-09-07,7356,Garon,2009-02-03,within an hour,100,100,t,Bedford-Stuyvesant,2.0,2.0,t,t,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,Private room,2,NaN,None,1.0,1.0,"[""Heating"", ""Kitchen"", ""Air conditioning"", ""Wi...",60.0,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,5,30,60,335,50,0,0,2009-05-28,2019-12-02,4.52,4.22,4.09,4.91,4.91,4.47,4.52,None,f,2,0,2,0,0.31,13.810959,0.0
3,45910,2022-09-07,204539,Mark,2010-08-17,within an hour,100,19,f,Ridgewood,7.0,7.0,t,t,Ridgewood,Queens,40.70309,-73.89963,Entire townhouse,Entire home/apt,16,NaN,2.5 baths,5.0,10.0,"[""Hair dryer"", ""Essentials"", ""Carbon monoxide ...",425.0,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,30,60,90,365,13,0,0,2012-01-03,2019-11-12,4.42,4.64,4.36,4.82,5.00,4.82,4.55,None,f,6,6,0,0,0.10,12.276712,2.5
4,5136,2022-09-07,7378,Rebecca,2009-02-03,None,None,33,f,Greenwood Heights,1.0,5.0,t,t,Sunset Park,Brooklyn,40.66265,-73.99454,Entire rental unit,Entire home/apt,4,NaN,1.5 baths,2.0,2.0,"[""Hair dryer"", ""Cable TV"", ""Refrigerator"", ""BB...",275.0,21,1125,21.0,21.0,1125.0,1125.0,21.0,1125.0,NaN,t,0,0,0,179,3,1,1,2014-01-02,2022-08-10,5.00,5.00,5.00,5.00,5.00,4.67,5.00,None,f,1,1,0,0,0.03,13.810959,1.5


Import geojsons

In [4]:
# lectura de geoson distritos
geosonNeighbourhoods = geopandas.read_file("../Data/neighbourhoods/neighbourhoods.geojson")
jsonGeoNeigh = json.load(open("../Data/neighbourhoods/neighbourhoods.geojson"))

# lectura de geoson barrios
bigJSONNeigh = json.load(open("../Data/neighbourhoods/bigneighbourhoods.geojson"))
geosonNeighbourhoodsBig = geopandas.read_file("../Data/neighbourhoods/bigneighbourhoods.geojson")

## Objetivo: Maximizar Rentabilidad

## Graficos a enseñar en Dash

### Tipo de habitación

## Distribución de precios 

*Price cap $1300*

Distribución de precios globales

Distribucion de precios por barrios